In [91]:
import streamlit as st
import pandas as pd
import networkx as nx
from pyvis.network import Network
import streamlit.components.v1 as components
from difflib import SequenceMatcher
import numpy as np

In [92]:
df = pd.read_csv('data\proximidade1.csv')

<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\joaof\AppData\Local\Temp\ipykernel_15936\1790388035.py:1: SyntaxWarning: invalid escape sequence '\p'
  df = pd.read_csv('data\proximidade1.csv')


In [93]:
# Manter o nome do respondente
nomes_respondentes = df.iloc[:, 1].astype(str).values

# Filtrar apenas as colunas relevantes (remover coluna de carimbo de data/hora)
df = df.iloc[:, 1:]

# Ajustar os nomes das colunas para obter apenas os nomes das pessoas
df.columns = df.columns.str.extract(r'Indique o nível de proximidade com (.*)')[0].str.strip()

# Preencher valores ausentes com 5 (assumindo que a ausência de resposta significa uma proximidade neutra)
df = df.fillna(5)

df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric)

In [94]:
df

,NaN,Artur Sousa Araujo,Beatriz Guedes,Bruno Martins Danziato,Eduardo Aronovich,Enrico Liberbaum,Franciele Taís,Gabriel Brana,Geovanna Felix,Gustavo Tironi,...,Lucas Coelho,Luis Felipe Alcantra,Luna Costa,Maria Antonia Botelho,Patrick Siles,Pedro Hajdu,Rafael Nunes,Rafaela Perlingeiro,Sarah Latouff,Victor Madsen
0,Maria Botelho,3,1,3.0,5.0,4,1,1.0,5,4,...,5,5.0,3.0,5.0,2.0,1,1,3.0,4.0,4
1,Lucas Coelho Silva,5,2,5.0,4.0,2,3,4.0,2,5,...,5,3.0,5.0,4.0,3.0,2,3,3.0,2.0,1
2,João Pedro Jerônimo de Oliveira,3,1,4.0,4.0,1,1,2.0,1,4,...,5,1.0,5.0,1.0,3.0,3,1,1.0,1.0,1
3,Gabriel Figueiredo Brana,3,1,5.0,4.0,2,5,5.0,4,3,...,5,4.0,3.0,2.0,4.0,2,5,1.0,1.0,3
4,João Pedro Fortes Soares,3,2,5.0,5.0,2,3,4.0,4,4,...,5,3.0,5.0,1.0,5.0,3,2,5.0,3.0,2
5,Rafaela Perlingeiro,3,2,5.0,4.0,1,2,5.0,5,5,...,4,3.0,5.0,1.0,2.0,3,3,5.0,2.0,1
6,João Felipe Vilas Boas,5,3,5.0,5.0,3,5,5.0,5,5,...,5,5.0,5.0,5.0,4.0,4,4,4.0,5.0,4
7,Luis Felipe Alcântara,1,1,2.0,1.0,1,2,4.0,2,3,...,2,5.0,2.0,3.0,1.0,1,1,4.0,4.0,1
8,Arthur de Sousa Araujo Ribeiro,5,1,4.0,2.0,1,1,2.0,1,3,...,5,1.0,3.0,1.0,2.0,2,1,1.0,2.0,1
9,Isaías Gouvêa Gonçalves,2,1,5.0,3.0,2,3,3.0,2,4,...,5,1.0,3.0,5.0,3.0,1,1,1.0,2.0,3


In [95]:
# Filtrar apenas as colunas relevantes (remover colunas de carimbo de data/hora e nome do respondente)
df

nomes_colunas = df.iloc[:, 0].values

nomes_linhas = df.columns.values
nomes_linhas = nomes_linhas[1:] # remover nan que está localizado na primeira posição

In [96]:
nomes_linhas

array(['Artur Sousa Araujo', 'Beatriz Guedes', 'Bruno Martins Danziato',
       'Eduardo Aronovich', 'Enrico Liberbaum', 'Franciele Taís',
       'Gabriel Brana', 'Geovanna Felix', 'Gustavo Tironi',
       'Helouise Mattjie', 'Isaias Gouvea', 'João Pedro Jeronimo',
       'João Pedro Soares', 'João Vicente Campos', 'João Vilas',
       'Luana Wajnberg', 'Lucas Coelho', 'Luis Felipe Alcantra',
       'Luna Costa', 'Maria Antonia Botelho', 'Patrick Siles',
       'Pedro Hajdu', 'Rafael Nunes', 'Rafaela Perlingeiro',
       'Sarah Latouff', 'Victor Madsen'], dtype=object)

In [97]:
nomes_colunas

array(['Maria Botelho', 'Lucas Coelho Silva',
       'João Pedro Jerônimo de Oliveira', 'Gabriel Figueiredo Brana',
       'João Pedro Fortes Soares', 'Rafaela Perlingeiro ',
       'João Felipe Vilas Boas ', 'Luis Felipe Alcântara',
       'Arthur de Sousa Araujo Ribeiro', 'Isaías Gouvêa Gonçalves ',
       'Helouise ', 'Rafael Costa de Menezes Nunes ', 'Bruno',
       'Geovanna Felix', 'Patrick Siles', 'Victor Madsen de O L Pinto',
       'Luna Costa Pereira', 'Beatriz Ferreira Guedes', 'sarah lattouf ',
       'Gustavo', 'Franciele Taís Willig', 'Luana Wajnberg',
       'João Vicente Campos Santos ', 'Enrico Liberbaum', 'Pedro Hajdu',
       'Eduardo Aronovich '], dtype=object)

In [98]:
def calcular_similaridade(nome1, nome2):
    # Verifica se ambos os valores são strings e não NaN
    if isinstance(nome1, str) and isinstance(nome2, str):
        return SequenceMatcher(None, nome1, nome2).ratio()
    return 0  # Retorna 0 para entradas inválidas

In [99]:
dicionario_correspondencias = {}

# Para cada nome em 'nomes_colunas', encontrar o nome em 'nomes_linhas' com maior similaridade
for nome_coluna in nomes_colunas:
    maior_similaridade = 0
    nome_mais_proximo = ''
    
    for nome_linha in nomes_linhas:
        # Calcula a similaridade se ambos forem válidos
        similaridade = calcular_similaridade(nome_coluna, nome_linha)
        if similaridade > maior_similaridade:
            maior_similaridade = similaridade
            nome_mais_proximo = nome_linha
    
    dicionario_correspondencias[nome_coluna] = nome_mais_proximo

dicionario_correspondencias["Bruno"] = "Bruno Martins Danziato"
# Exibir o dicionário final
print(dicionario_correspondencias)

{'Maria Botelho': 'Maria Antonia Botelho', 'Lucas Coelho Silva': 'Lucas Coelho', 'João Pedro Jerônimo de Oliveira': 'João Pedro Jeronimo', 'Gabriel Figueiredo Brana': 'Gabriel Brana', 'João Pedro Fortes Soares': 'João Pedro Soares', 'Rafaela Perlingeiro ': 'Rafaela Perlingeiro', 'João Felipe Vilas Boas ': 'João Vilas', 'Luis Felipe Alcântara': 'Luis Felipe Alcantra', 'Arthur de Sousa Araujo Ribeiro': 'Artur Sousa Araujo', 'Isaías Gouvêa Gonçalves ': 'Isaias Gouvea', 'Helouise ': 'Helouise Mattjie', 'Rafael Costa de Menezes Nunes ': 'Rafael Nunes', 'Bruno': 'Bruno Martins Danziato', 'Geovanna Felix': 'Geovanna Felix', 'Patrick Siles': 'Patrick Siles', 'Victor Madsen de O L Pinto': 'Victor Madsen', 'Luna Costa Pereira': 'Luna Costa', 'Beatriz Ferreira Guedes': 'Beatriz Guedes', 'sarah lattouf ': 'Sarah Latouff', 'Gustavo': 'Gustavo Tironi', 'Franciele Taís Willig': 'Franciele Taís', 'Luana Wajnberg': 'Luana Wajnberg', 'João Vicente Campos Santos ': 'João Vicente Campos', 'Enrico Liberbau

In [100]:
df

,NaN,Artur Sousa Araujo,Beatriz Guedes,Bruno Martins Danziato,Eduardo Aronovich,Enrico Liberbaum,Franciele Taís,Gabriel Brana,Geovanna Felix,Gustavo Tironi,...,Lucas Coelho,Luis Felipe Alcantra,Luna Costa,Maria Antonia Botelho,Patrick Siles,Pedro Hajdu,Rafael Nunes,Rafaela Perlingeiro,Sarah Latouff,Victor Madsen
0,Maria Botelho,3,1,3.0,5.0,4,1,1.0,5,4,...,5,5.0,3.0,5.0,2.0,1,1,3.0,4.0,4
1,Lucas Coelho Silva,5,2,5.0,4.0,2,3,4.0,2,5,...,5,3.0,5.0,4.0,3.0,2,3,3.0,2.0,1
2,João Pedro Jerônimo de Oliveira,3,1,4.0,4.0,1,1,2.0,1,4,...,5,1.0,5.0,1.0,3.0,3,1,1.0,1.0,1
3,Gabriel Figueiredo Brana,3,1,5.0,4.0,2,5,5.0,4,3,...,5,4.0,3.0,2.0,4.0,2,5,1.0,1.0,3
4,João Pedro Fortes Soares,3,2,5.0,5.0,2,3,4.0,4,4,...,5,3.0,5.0,1.0,5.0,3,2,5.0,3.0,2
5,Rafaela Perlingeiro,3,2,5.0,4.0,1,2,5.0,5,5,...,4,3.0,5.0,1.0,2.0,3,3,5.0,2.0,1
6,João Felipe Vilas Boas,5,3,5.0,5.0,3,5,5.0,5,5,...,5,5.0,5.0,5.0,4.0,4,4,4.0,5.0,4
7,Luis Felipe Alcântara,1,1,2.0,1.0,1,2,4.0,2,3,...,2,5.0,2.0,3.0,1.0,1,1,4.0,4.0,1
8,Arthur de Sousa Araujo Ribeiro,5,1,4.0,2.0,1,1,2.0,1,3,...,5,1.0,3.0,1.0,2.0,2,1,1.0,2.0,1
9,Isaías Gouvêa Gonçalves,2,1,5.0,3.0,2,3,3.0,2,4,...,5,1.0,3.0,5.0,3.0,1,1,1.0,2.0,3


In [101]:
# Substituir os valores da primeira coluna de acordo com o dicionário
df.columns.values[0] = 'Nomes'

In [102]:
import pandas as pd

# Supondo que o DataFrame já esteja carregado no df
# Verifique o nome das colunas e renomeie a primeira coluna sem nome

# Se a primeira coluna estiver sem nome (ou com NaN), renomeie para 'Nomes'
df.columns = df.columns.to_list()  # Transforma os nomes de colunas em uma lista
df.columns.values[0] = 'Nomes'  # Renomeia a primeira coluna

# Verifique se a coluna foi renomeada corretamente
print(df.columns)  # Confirme que a primeira coluna agora se chama 'Nomes'

# Substituir os valores da coluna 'Nomes' com base no dicionário de correspondências
df['Nomes'] = df['Nomes'].map(lambda x: dicionario_correspondencias.get(x, x))

# Exibir o dataframe atualizado
print(df)


Index(['Nomes', 'Artur Sousa Araujo', 'Beatriz Guedes',
       'Bruno Martins Danziato', 'Eduardo Aronovich', 'Enrico Liberbaum',
       'Franciele Taís', 'Gabriel Brana', 'Geovanna Felix', 'Gustavo Tironi',
       'Helouise Mattjie', 'Isaias Gouvea', 'João Pedro Jeronimo',
       'João Pedro Soares', 'João Vicente Campos', 'João Vilas',
       'Luana Wajnberg', 'Lucas Coelho', 'Luis Felipe Alcantra', 'Luna Costa',
       'Maria Antonia Botelho', 'Patrick Siles', 'Pedro Hajdu', 'Rafael Nunes',
       'Rafaela Perlingeiro', 'Sarah Latouff', 'Victor Madsen'],
      dtype='object')
                     Nomes  Artur Sousa Araujo  Beatriz Guedes  \
0    Maria Antonia Botelho                   3               1   
1             Lucas Coelho                   5               2   
2      João Pedro Jeronimo                   3               1   
3            Gabriel Brana                   3               1   
4        João Pedro Soares                   3               2   
5      Rafaela Per

In [103]:
df

,Nomes,Artur Sousa Araujo,Beatriz Guedes,Bruno Martins Danziato,Eduardo Aronovich,Enrico Liberbaum,Franciele Taís,Gabriel Brana,Geovanna Felix,Gustavo Tironi,...,Lucas Coelho,Luis Felipe Alcantra,Luna Costa,Maria Antonia Botelho,Patrick Siles,Pedro Hajdu,Rafael Nunes,Rafaela Perlingeiro,Sarah Latouff,Victor Madsen
0,Maria Antonia Botelho,3,1,3.0,5.0,4,1,1.0,5,4,...,5,5.0,3.0,5.0,2.0,1,1,3.0,4.0,4
1,Lucas Coelho,5,2,5.0,4.0,2,3,4.0,2,5,...,5,3.0,5.0,4.0,3.0,2,3,3.0,2.0,1
2,João Pedro Jeronimo,3,1,4.0,4.0,1,1,2.0,1,4,...,5,1.0,5.0,1.0,3.0,3,1,1.0,1.0,1
3,Gabriel Brana,3,1,5.0,4.0,2,5,5.0,4,3,...,5,4.0,3.0,2.0,4.0,2,5,1.0,1.0,3
4,João Pedro Soares,3,2,5.0,5.0,2,3,4.0,4,4,...,5,3.0,5.0,1.0,5.0,3,2,5.0,3.0,2
5,Rafaela Perlingeiro,3,2,5.0,4.0,1,2,5.0,5,5,...,4,3.0,5.0,1.0,2.0,3,3,5.0,2.0,1
6,João Vilas,5,3,5.0,5.0,3,5,5.0,5,5,...,5,5.0,5.0,5.0,4.0,4,4,4.0,5.0,4
7,Luis Felipe Alcantra,1,1,2.0,1.0,1,2,4.0,2,3,...,2,5.0,2.0,3.0,1.0,1,1,4.0,4.0,1
8,Artur Sousa Araujo,5,1,4.0,2.0,1,1,2.0,1,3,...,5,1.0,3.0,1.0,2.0,2,1,1.0,2.0,1
9,Isaias Gouvea,2,1,5.0,3.0,2,3,3.0,2,4,...,5,1.0,3.0,5.0,3.0,1,1,1.0,2.0,3


In [105]:
import pandas as pd

# Suponha que o DataFrame df já esteja carregado conforme descrito

# Capturar os nomes da coluna 'Nomes' e as colunas restantes (nomes das pessoas)
nomes_linhas = df['Nomes'].values  # Valores da coluna 'Nomes'
nomes_colunas = df.columns[1:]     # Nomes das colunas a partir da segunda

# Criar um novo DataFrame vazio para a matriz simétrica, incluindo os próprios nomes como colunas
matriz_simetrica = pd.DataFrame(index=nomes_linhas, columns=nomes_linhas)

# Preencher a matriz com os valores correspondentes
for i, nome_linha in enumerate(nomes_linhas):
    for nome_coluna in nomes_linhas:
        # Se os nomes coincidem (diagonal principal), o valor é 1 ou pode ser outro valor se desejado
        if nome_linha == nome_coluna:
            matriz_simetrica.at[nome_linha, nome_coluna] = 1  # ou outro valor desejado
        else:
            # Caso contrário, preencher com os valores originais do DataFrame
            valor = df.loc[df['Nomes'] == nome_linha, nome_coluna].values[0]
            matriz_simetrica.at[nome_linha, nome_coluna] = valor

matriz_simetrica


,Maria Antonia Botelho,Lucas Coelho,João Pedro Jeronimo,Gabriel Brana,João Pedro Soares,Rafaela Perlingeiro,João Vilas,Luis Felipe Alcantra,Artur Sousa Araujo,Isaias Gouvea,...,Luna Costa,Beatriz Guedes,Sarah Latouff,Gustavo Tironi,Franciele Taís,Luana Wajnberg,João Vicente Campos,Enrico Liberbaum,Pedro Hajdu,Eduardo Aronovich
Maria Antonia Botelho,1,5,1,1.0,1,3.0,5,5.0,3,4,...,3.0,1,4.0,4,1,4,4,4,1,5.0
Lucas Coelho,4.0,1,5,4.0,5,3.0,5,3.0,5,4,...,5.0,2,2.0,5,3,2,5,2,2,4.0
João Pedro Jeronimo,1.0,5,1,2.0,3,1.0,4,1.0,3,4,...,5.0,1,1.0,4,1,1,4,1,3,4.0
Gabriel Brana,2.0,5,3,1,5,1.0,4,4.0,3,2,...,3.0,1,1.0,3,5,1,4,2,2,4.0
João Pedro Soares,1.0,5,4,4.0,1,5.0,4,3.0,3,2,...,5.0,2,3.0,4,3,2,4,2,3,5.0
Rafaela Perlingeiro,1.0,4,1,5.0,5,1,2,3.0,3,1,...,5.0,2,2.0,5,2,5,5,1,3,4.0
João Vilas,5.0,5,5,5.0,5,4.0,1,5.0,5,4,...,5.0,3,5.0,5,5,5,5,3,4,5.0
Luis Felipe Alcantra,3.0,2,1,4.0,3,4.0,4,1,1,1,...,2.0,1,4.0,3,2,3,4,1,1,1.0
Artur Sousa Araujo,1.0,5,3,2.0,3,1.0,4,1.0,1,1,...,3.0,1,2.0,3,1,1,3,1,2,2.0
Isaias Gouvea,5.0,5,5,3.0,3,1.0,3,1.0,2,1,...,3.0,1,2.0,4,3,1,4,2,1,3.0


In [107]:
output_path = 'data/matriz_simetrica.csv'

# Salvar a matriz em um arquivo CSV
matriz_simetrica.to_csv(output_path)

# Retornar o caminho onde o arquivo foi salvo
output_path

'data/matriz_simetrica.csv'